[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

LangChain에서는 대규모 언어 모델(LLM) 애플리케이션을 쉽게 구축할 수 있도록 하는 것을 목표로 합니다. 구축 가능한 LLM 애플리케이션 유형 중 하나는 에이전트입니다. 에이전트 구축은 이전에는 불가능했던 다양한 작업을 자동화할 수 있기 때문에 많은 관심을 받고 있습니다. 

하지만 실제로는 이러한 작업을 안정적으로 수행하는 시스템을 구축하는 것이 매우 어렵습니다. 사용자와 함께 에이전트를 실제 운영 환경에 적용해 본 경험을 통해, 더 많은 제어 기능이 필요하다는 점을 깨달았습니다. 에이전트가 항상 특정 도구를 먼저 호출하도록 하거나 상태에 따라 다른 프롬프트를 사용하도록 설정해야 할 수 있습니다. 

이 문제를 해결하기 위해 우리는 에이전트 및 다중 에이전트 애플리케이션 구축을 위한 프레임워크인 [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview)를 개발했습니다. LangChain 패키지와 별개로, LangGraph의 핵심 설계 철학은 개발자가 실제 시스템의 복잡성에 적합한 정밀도와 제어력을 에이전트 워크플로우에 더할 수 있도록 돕는 것입니다.

## Course Structure

본 과정은 일련의 모듈로 구성되어 있으며, 각 모듈은 LangGraph와 관련된 특정 주제에 초점을 맞춥니다. 각 모듈별로 폴더가 생성되며, 해당 폴더에는 일련의 노트북이 포함되어 있습니다. 각 노트북에는 개념을 단계별로 안내하는 동영상이 함께 제공되지만, 노트북 자체도 독립적으로 작동합니다. 즉, 설명이 포함되어 있어 동영상 없이도 볼 수 있습니다. 각 모듈 폴더에는 `studio` 폴더도 포함되어 있으며, 여기에는 LangGraph 애플리케이션 구축용 IDE인 [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio)에 로드할 수 있는 그래프 세트가 들어 있습니다.

## Chat models

이 과정에서는 메시지 시퀀스를 입력으로 받아 메시지를 출력으로 반환하는 채팅 모델을 사용할 것입니다. LangChain은 [타사 통합](https://docs.langchain.com/oss/python/integrations/chat)을 통해 다양한 모델을 지원합니다. 기본적으로 본 과정에서는 널리 사용되며 성능이 우수한 [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai)를 사용할 예정입니다. 

In [1]:
# %%capture --no-stderr
# %pip install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily

In [2]:
# import os, getpass

# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")

from dotenv import load_dotenv

load_dotenv("../../.env")

True

챗 모델에는 설정 가능한 [몇 가지 표준 매개변수](https://docs.langchain.com/oss/python/langchain/models#parameters)가 있습니다. 가장 흔히 사용되는 두 가지는 다음과 같습니다:

* `model`: 모델 이름
* `temperature`: 샘플링 온도

`Temperature`는 모델 출력의 무작위성 또는 창의성을 제어합니다. 낮은 온도(0에 가까울수록)는 더 결정론적이고 집중된 출력을 생성합니다. 정확성이나 사실적인 응답이 필요한 작업에 적합합니다. 높은 온도(1에 가까울수록)는 창의적인 작업이나 다양한 응답 생성에 적합합니다. 

In [3]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

LangChain의 채팅 모델은 여러 [기본 메서드](https://reference.langchain.com/python/langchain_core/runnables)를 제공합니다. 주로 다음과 같은 메서드를 사용할 것입니다:

* [stream](https://docs.langchain.com/oss/python/langchain/models#stream): 응답을 청크 단위로 스트리밍하여 반환
* [invoke](https://docs.langchain.com/oss/python/langchain/models#invoke): 입력에 대해 체인을 호출

또한 앞서 언급했듯이, 채팅 모델은 [messages](https://docs.langchain.com/oss/python/langchain/messages)를 입력으로 받습니다. 메시지는 발신자를 나타내는 역할(role)과 내용(content) 속성을 가집니다. 이에 대해서는 나중에 자세히 다루겠지만, 여기서는 기본적인 내용만 살펴보겠습니다.

In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="안녕하세요.", name="유광명")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ed36a96f3d', 'id': 'chatcmpl-D9k41O6YBh7VEa9tJMF9th7DkmRZ3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c649a-cf7f-7930-aca5-73cc98889f29-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

모델을 실행하면 `AIMessage` 응답을 받습니다.   
문자열로 채팅 모델을 호출할 수 있다는 점에 유의하세요.   
문자열이 입력으로 전달되면 `HumanMessage`로 변환된 후 기본 모델로 전달됩니다.

In [5]:
gpt4o_chat.invoke("안녕하세요")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ed36a96f3d', 'id': 'chatcmpl-D9k59VMsHHorVMt9osmLSq7Y4CGN5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c649b-e199-7533-bcb0-98916c158313-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Search Tools

README에서 [Tavily](https://tavily.com/)도 확인할 수 있습니다. 이는 대규모 언어 모델(LLM)과 RAG(Retrieval-Augmented Generation)에 최적화된 검색 엔진으로, 효율적이고 신속하며 지속적인 검색 결과를 목표로 합니다. 앞서 언급했듯이 가입이 간편하며 무료 이용권도 넉넉하게 제공됩니다. 일부 강의(모듈 4)에서는 기본적으로 Tavily를 사용하지만, 코드를 직접 수정하고 싶다면 다른 검색 도구도 물론 사용할 수 있습니다.

In [6]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "랭그래프가 뭐야?"})
search_docs = data.get("results", data)

In [7]:
search_docs

[{'url': 'https://brunch.co.kr/@aideveloper/132',
  'title': '23화 22. LangGraph 기본과 응용 - 브런치',
  'content': 'LangGraph(랭그래프)는 LangChain(랭체인)을 기반으로 하는 에이전트 프레임워크입니다. LangGraph(랭그래프)의 주요 특징은 복잡한 제어구조를 가진 워크플로우를 유연하게 표현할 수 있다는 점입니다. 반면에 LangGraph(랭그래프)는 체인뿐만 아니라 일반적인 그래프 구조를 사용하여 복잡한 제어구조를 표현할 수 있습니다. LangGraph(랭그래프)는 노드와 엣지를 결합하여 멀티 에이전트 구현에 적합한 워크플로우를 표현합니다. 예를 들어, 아래 그림은 LangGraph(랭그래프)를 사용하여 소프트웨어 개발팀을 멀티에이전트로 구현하는 그래프입니다. 개발자는 LangGraph(랭그래프)를 사용하여 다양한 워크플로우를 그래프로 정의할 수 있습니다. LangGraph(랭그래프)는 에이전트 시스템에 국한되지 않고, 범용 워크플로우를 정의하는데 사용할 수 있습니다. 이 때문에 LangGraph(랭그래프)의 그래프를 상태그래프(StateGraph)라고도 합니다. LangGraph(랭그래프)는 각 노드의 처리를 Python함수 또는 Runnable객체로 정의합니다. 그 경우 LangGraph(랭그래프)의 사양에 따라 상태를 인수로 받아야 합니다. LangGraph(랭그래프)에서는 조건에 따라 노드간의 전환을 제어할 수 있습니다. StateGraph클래스를 사용하여 이 상태 그래프를 정의합니다. 이번에는 LangGraph(랭그래프)를 사용하여 대표적인 에이전트 아키텍처를 구현하는 방법을 소개합니다. LangGraph(랭그래프)를 사용하여 자연어 쉘 인터페이스를 사용하여 단일 에이전트를 구축하는 방법을 소개합니다. 이제 LangGraph(랭그래프)를 사용하여 이 자연어 쉘 인터페이스를 구현해 보겠습니다. LangGraph(랭그래프)를 사용하여 수평 아키텍처에서 에이전트